In [1]:
# USER-SPECFIC: The folder path holding CWatM simulations 
from netCDF4 import Dataset, num2date
import numpy as np
import numpy.ma as ma

if not 'output_folder' in locals():  
    output_folder = r'C:\Users\smilovic\OneDrive - IIASA\Design\Assignment Watercycle\Output_eLVB'

if not 'outlets' in locals():
    # The location coordinates of the final cell in the basin, the discharge point, 
    latitude = 17.3875 #51.84 
    longitude = 75.89583 #6.11
    #latitude = 2753500 #2018532 #51.84 
    #longitude = 4545500 #3519921  #6.11
    
    outlets = [(latitude, longitude)]

if not 'Modflow' in locals():
    Modflow = False

if not 'limitAbstraction' in locals():
    limitAbstraction = True
    
if not 'consumption' in locals():
    consumption = True
    
if not 'withdrawals' in locals():
    withdrawals = False
    
if not 'withCrops' in locals():
    withCrops = False
    numberOfCrops = 18
    
if not 'regularGrid' in locals():
    regularGrid = False

In [2]:
# ['CWatm variable name', 'Label', 'Unit', Input/Output/Storage']

# There are three different types of Units: 
# 1. Spatially districuted values in M
# 2. Spatially distributed values in M3
# 3. Summed total for the basin, where each cell holds the total sum for the basin

if Modflow == False:
    Vars = [
        ['storGroundwater',   'Groundwater',       'M',      'Storage'],
        ['storGroundwater',   'Groundwater',       'M',      'Storage_GW'],
        ['sum_capRiseFromGW',         'Capillary rise',         'M',      'Input_GW'],
        ['sum_capRiseFromGW',         'Capillary rise',         'M',      'Output_GW'], #February 26
        ['sum_capRiseFromGW',         'Capillary rise',         'M',      'Output_Soil'],
        ['sum_capRiseFromGW',                'Capillary rise',         'M',  'Input_Soil']]
        
    if limitAbstraction == True:
        Vars.extend([['nonFossilGroundwaterAbs', 'GW pumping', 'M', 'Output_GW']]) #for limitAbstraction
        Vars.extend([['nonFossilGroundwaterAbs', 'GW pumping', 'M', 'Input_Consumption']]) #for limitAbstraction
    else:
        Vars.extend([['unmetDemand', 'Fossil water', 'M', 'Input']]) # T=to test
        Vars.extend([['unmetDemand', 'Fossil water', 'M', 'Input_GW']]) # to test
        Vars.extend([['pot_GroundwaterAbstract',                 'GW requested', 'M',  'Input_Consumption']])
        Vars.extend([['pot_GroundwaterAbstract', 'GW pumping', 'M', 'Output_GW']])

else:
    Vars = [
        ['leakage',               'Canal leakage',                  'M3',   'Output_LR'],
        ['groundwater_storage_available',            'Groundwater',       'M', 'Storage'],
        ['groundwater_storage_available',            'Groundwater',       'M', 'Storage_GW'],
        ['capillar',         'Capillary rise',         'M',      'Output_GW'],
        ['capillar',         'Capillary rise',         'M',      'Input_Soil'],
        ['modfPumpingM_actual',     'GW pumping actual','M', 'Output_GW'],
        ['Pumping_daily', 'GW requested', 'M',  'Input_Consumption'],
        ['riverbedExchangeM',       'Riverbed exchange',      'M',  'Input_Soil'],
        ['leakageIntoRunoff', 'Canal leakage (Runoff)', 'M', 'Input_Soil'],
        ['leakageIntoGw', 'Canal leakage (GW)', 'M', 'Input_Soil'],
        ['riverbedExchangeM', 'Riverbed exchange', 'M', 'Input_GW'],
        ['lakebedExchangeM', 'Lakebed exchange', 'M', 'Input_GW'],
        ['lakebedExchangeM', 'Lakebed exchange', 'M', 'Output_LR'],
        ['lakebedExchangeM', 'Lakebed exchange', 'M', 'Input_Soil'],
        ['leakageIntoGw', 'Canal leakage (GW)', 'M', 'Input_GW']]

smallLakes = False
if smallLakes:
    Vars.extend([
        ['smallevapWaterBody',        'Small Water bodies evaporation',           'M',      'Output'],
        ['smalllakeStorage',        'Small Water bodies storage',               'M3',     'Storage']
    ])
    
Vars.extend([
    # General Water Balance
    ['Rain',                  'Rain',                               'M',      'Input'],
    ['Snow',                  'Snow',                               'M',      'Input'],
    #['Precipitation',                  'Precipitation',                               'M',      'Input'],
    ['totalET',               'Evapotranspiration',          'M',      'Output'],
    ['EvapoChannel',          'Channel evap.',                'M3',     'Output'],
    ['EvapWaterBodyM',        'Lake evap.',           'M',      'Output'],
    #['smallevapWaterBody',        'Small Water bodies evaporation',           'M',      'Output'],
    ['act_nonIrrConsumption', 'non-Irrigation consumption',         'M',      'Output'],
    #['smalllakeStorage',        'Small Water bodies storage',               'M3',     'Storage'],
    ['channelStorage',        'Rivers',                    'M3',     'Storage'],
    ['lakeResStorage',        'Lakes',               'M3',     'Storage'],
    ['totalSto',              'Soil*',           'M',      'Storage'],
    ['unmet_lost',        'Non-lost fossil water',           'M',      'Output'],


    ['sum_actTransTotal',       'Transpiration',                'M',    'Output_ET'],
    ['actTransTotal_forest',    'Forest',       'M',    'Output_Trans'],
    ['actTransTotal_grasslands','Grasslands',   'M',    'Output_Trans'],
    ['actTransTotal_paddy',     'Paddy',        'M',    'Output_Trans'],
    ['actTransTotal_nonpaddy',  'non-Paddy',    'M',    'Output_Trans'],


    ['sum_actBareSoilEvap',     'Bare soil evap.',        'M',    'Output_ET'],
    ['sum_interceptEvap',       'Intercept evap.',        'M',    'Output_ET'],
    ['sum_openWaterEvap',       'Open water evap.',       'M',    'Output_ET'],
    ['addtoevapotrans',         'Evap. application loss',   'M',    'Output_ET'],
    ['snowEvap',         'Snow evap.',   'M',    'Output_ET'],
    

    # Lakes and Reservoirs Water Balance
    ['lakeResInflowM',        'Lake inflow',               'M',    'Input_LR'],
    ['EvapWaterBodyM',        'Lake evap.',          'M',    'Output_LR'],
    ['act_bigLakeResAbst',    'Delivered lake','M',    'Output_LR'],
    #['leakage',               'Canal leakage',                  'M3',   'Output_LR'],
    ['lakeResOutflowM',       'Lake discharge',              'M',    'Output_LR'],
    ['lakeResStorage',        'Lake storage',              'M3',    'Storage_LR'],

    # Groundwater Water Balance
    #['leakage',          'Canal leakage',          'M3',     'Input_GW'],
    ['baseflow',         'Baseflow',               'M',      'Output_GW'],
    ['prefFlow_GW', 'Preferential flow', 'M', 'Input_GW'],
    ['perc3toGW_GW', 'Soil percolation', 'M', 'Input_GW'],


    # Soil Water Balance
    #['Rain',                    'Rain',                   'M',  'Input_Soil'],
    #['Precipitation',                  'Precipitation',                               'M',      'Input_Soil'],
    ['Rain',                  'Rain',                               'M',      'Input_Soil'],
    ['Snow',                  'Snow',                               'M',      'Input_Soil'],
    ['act_totalIrrConsumption', 'Effective irrigation',        'M',   'Input_Soil'],

    ['sum_actTransTotal',       'Transpiration',          'M',  'Output_Soil'],
    ['sum_actBareSoilEvap',     'Bare soil evap.',  'M',  'Output_Soil'],
    ['sum_interceptEvap',       'Intercept evap.',  'M',  'Output_Soil'],
    ['sum_openWaterEvap',       'Open water evap.', 'M',  'Output_Soil'],
    ['sum_runoff',              'Runoff',                 'M', 'Output_Soil'],
    ['sum_gwRecharge',          'GW recharge',            'M',  'Output_Soil'],
    ['snowEvap',         'Snow evap.',   'M',    'Output_Soil'],
    #['SnowCover',                'Snow storage','M',  'Storage_Soil'],
    ['totalSto',                'Soil*','M',  'Storage_Soil']])

In [3]:
# Loading the data

import numpy.ma as ma
nc_simulated = Dataset(output_folder + '/discharge_daily.nc', 'r')
Dates_simulation = num2date(nc_simulated.variables['time'][:], units=nc_simulated.variables['time'].units)

# WB = {Inputs:  [ ['Rain', [rain_value_day0, ..., rain_value_dayLast]] ],
#       Outputs: [ ['Evapotranspiration (soil)', [ET_value_day0, ..., ET_value_dayLast]] 
#                  ['Channel evaporation', [ChannelE_value_day0, ..., ChannelE_value_dayLast]], ... ]}

keys = [i[-1] for i in Vars]
WB = {key: [] for key in keys}

cellArea = Dataset(output_folder +'/'+ 'cellArea_totalend.nc', 'r').variables['cellArea_totalend'][:,:]
cellAreaC = ma.compressed(cellArea)

cellAreaSum = np.sum(ma.compressed(cellAreaC))

#limitAbstraction = True
print("Loading the following variables:")
for var in Vars:
    print(var[0])
    nc_filename = output_folder +'/'+ var[0] + '_daily.nc'
    temp = Dataset(nc_filename, 'r').variables[var[0]][:,:,:]
    Daily_basinSum = []
    
    for d in range(len(Dates_simulation)):
        
        if var[2] == 'M':
            daily_basinSum = np.sum(np.multiply(ma.compressed(temp[d,:,:]), cellAreaC))
            if limitAbstraction == True and var[0]=='unmetDemand': 
                daily_basinSum *=0
        elif var[2] == 'M3':
            daily_basinSum = np.sum(ma.compressed(temp[d,:,:]))
        else:
            #All cells hold the same total-value for the basin
            daily_basinSum = ma.compressed(temp[d,:,:])[0]
            
        Daily_basinSum.append(daily_basinSum)
    
    WB[var[-1]].append([var[1], np.array(Daily_basinSum)])

Loading the following variables:
storGroundwater
storGroundwater
sum_capRiseFromGW
sum_capRiseFromGW
sum_capRiseFromGW
sum_capRiseFromGW
nonFossilGroundwaterAbs
nonFossilGroundwaterAbs
Rain
Snow
totalET
EvapoChannel
EvapWaterBodyM
act_nonIrrConsumption
channelStorage
lakeResStorage
totalSto
unmet_lost
sum_actTransTotal
actTransTotal_forest
actTransTotal_grasslands
actTransTotal_paddy
actTransTotal_nonpaddy
sum_actBareSoilEvap
sum_interceptEvap
sum_openWaterEvap
addtoevapotrans
snowEvap
lakeResInflowM
EvapWaterBodyM
act_bigLakeResAbst
lakeResOutflowM
lakeResStorage
baseflow
prefFlow_GW


FileNotFoundError: [Errno 2] No such file or directory: b'C:\\Users\\smilovic\\OneDrive - IIASA\\Design\\Assignment Watercycle\\Output_eLVB/prefFlow_GW_daily.nc'